In [4]:
import numpy as np
import matplotlib.pyplot as plt
import lifesim


In [5]:
#LIFEsim is capable of taking an artificial exoplanet catalog input from P-Pop, calculate the signal-to-noise ratio for each planet and distribute the observation time available in the search phase to observe and detect an optimal number of exoplanets

# Create an instance to save all data and parameters in a single location
bus = lifesim.Bus()

# Set the parameters to the baseline case by running
bus.data.options.set_scenario('baseline')


In [8]:
# A example synthetic planet population based on statistics from the Kepler mission can be downloaded from the P-Pop github page
filename = '/home/pujitha/PycharmProjects/lifesim/TestPlanetPopulation.txt'

#P-Pop catalog can be loaded i
bus.data.catalog_from_ppop(filename)

Processed line 45884 of 45884


In [12]:
#an instance of the LIFEsim instrument module needs to be created
instrument = lifesim.Instrument(name='inst')


In [31]:
#To give any module access to the data and parameters used in a simulation, it needs to be connected to the bus
#bus.add_module(instrument)


In [19]:
#All modules needed for the instrument module to run need to be created. Create the module responsible for simulating transmission maps of a four-arm nulling interferometer and add it to the bus
transm = lifesim.TransmissionMap(name='transm')
bus.add_module(transm)

In [15]:
#create the modules for the simulation of the astrophysical noise sources and add them to the bus.
exozodi = lifesim.PhotonNoiseExozodi(name='exo')
bus.add_module(exozodi)
localzodi = lifesim.PhotonNoiseLocalzodi(name='local')
bus.add_module(localzodi)
star_leak = lifesim.PhotonNoiseStar(name='star')
bus.add_module(star_leak)

In [22]:
bus.connect(('inst', 'transm'))
bus.connect(('inst', 'exo'))
bus.connect(('inst', 'local'))
bus.connect(('inst', 'star'))

bus.connect(('star', 'transm'))

In [23]:
#The optimizer is responsible for distributing the available observing time onto the individual stars. Analogously to above, we have

opt = lifesim.Optimizer(name='opt')
bus.add_module(opt)
ahgs = lifesim.AhgsModule(name='ahgs')
bus.add_module(ahgs)

bus.connect(('transm', 'opt'))
bus.connect(('inst', 'opt'))
bus.connect(('opt', 'ahgs'))

In [24]:
#the signal-to-noise ratio needs to be calculated for every planet in the catalog. To do so, we run
instrument.get_snr()


100%|██████████| 1732/1732 [05:44<00:00,  5.02it/s]


In [ ]:
#Saving the Results

bus.data.export_catalog(output_path = '/home/pujitha/PycharmProjects/lifesim/filename.hdf5')
